# [E-04] Make a Lyricsht
### 멋진 작사가

# Step 1. 데이터 다운로드

~/aiffel/lyricist/data/lyrics 폴더

# Step 2. 데이터 읽어오기

In [1]:
import glob
import numpy as np
import tensorflow as tf
import os, re

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# Step 3. 데이터 정제

입력된 문장을 
1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 start, 끝에는 end를 추가합니다

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [17]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue 
    if len(sentence.split())< 3: continue #oh oh 같은 반복되는 불필요한 부분 제거 하고 싶다

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>']

In [18]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #단어장 크기
        filters=' ',
        oov_token="<unk>" #12000단어에 포함되지 못한 단어는 <unk>
    )

    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
        # 문장 길이를 15로 줄이기
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor.shape

[[   2   50    5 ...    0    0    0]
 [   2   17 2601 ...    0    0    0]
 [   2   35    7 ...   44    3    0]
 ...
 [   5   21   10 ...    9 1003    3]
 [  37   15 8893 ...  868  637    3]
 [   2    7   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f768159c0a0>


(167650, 15)

In [19]:
print(tensor[:3, :10])

[[   2   50    5   91  294   64   55   10  968 5924]
 [   2   17 2601  867    4    8   11 5925    6  332]
 [   2   35    7   37   15  164  281   28  298    4]]


In [20]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : to
10 : a


# Step 4. 평가 데이터셋 분리

sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리
- 단어장 크기: 7000 
- test_data 20%

In [21]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=7)
# src와 tgt
print(src_input[0])
print(tgt_input[0])

print(len(src_input))

[   2   50    5   91  294   64   55   10  968 5924    3    0    0    0]
[  50    5   91  294   64   55   10  968 5924    3    0    0    0    0]
167650


In [23]:
print("Source Train:", enc_train.shape)
print("Source Val:", enc_val.shape)
print("Target Train:", dec_train.shape)
print("Target Val:", dec_val.shape)

Source Train: (134120, 14)
Source Val: (33530, 14)
Target Train: (134120, 14)
Target Val: (33530, 14)


In [24]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

지금까지의 데이터셋을 생성하기 위해 텐서플로우에서의 데이터 전처리 과정

- 정규표현식을 이용한 corpus 생성
- tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환
- dataset 생성

tf.data.Dataset에서 제공하는 shuffle(), batch() 등 다양한 데이터셋 관련 기능을 손쉽게 이용할 수 있게 되었군요.

In [25]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

- embedding_size 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
- hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가?

In [26]:
for src_input, tgt_input in dataset.take(1): break


model(src_input)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.0470767e-05,  1.1893480e-04, -3.7375768e-05, ...,
         -1.6645015e-04,  2.0513535e-04,  9.3048009e-05],
        [-2.8775208e-05,  2.6061709e-04,  6.3643303e-05, ...,
         -5.0798384e-04,  2.9017526e-04,  3.9747465e-05],
        [ 2.1082172e-04,  3.0830229e-04,  1.1150116e-04, ...,
         -7.5830135e-04,  3.3282969e-04,  9.6566859e-05],
        ...,
        [ 2.4711963e-04, -4.3889860e-04,  7.7391957e-04, ...,
         -1.8966912e-03,  6.0240453e-04,  7.1113429e-04],
        [ 6.8360096e-05, -6.9753540e-04,  8.9338422e-04, ...,
         -1.9756514e-03,  5.9153530e-04,  9.8981475e-04],
        [-1.2169771e-04, -9.6565415e-04,  9.7905006e-04, ...,
         -2.0215425e-03,  5.5125490e-04,  1.2388235e-03]],

       [[ 1.0470767e-05,  1.1893480e-04, -3.7375768e-05, ...,
         -1.6645015e-04,  2.0513535e-04,  9.3048009e-05],
        [-1.4622266e-04,  1.5582706e-04, -7.8119476e-05, ...,
         -2.7786259e-04, 

In [27]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  18882560  
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_1 (Dense)              multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


# Step 5. 인공지능 만들기

In [28]:
#Loss
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
130/130 [==============================] - 66s 491ms/step - loss: 4.5263
Epoch 2/10
130/130 [==============================] - 64s 489ms/step - loss: 3.7868
Epoch 3/10
130/130 [==============================] - 63s 488ms/step - loss: 3.5702
Epoch 4/10
130/130 [==============================] - 64s 491ms/step - loss: 3.3709
Epoch 5/10
130/130 [==============================] - 63s 488ms/step - loss: 3.2301
Epoch 6/10
130/130 [==============================] - 63s 487ms/step - loss: 3.1183
Epoch 7/10
130/130 [==============================] - 63s 486ms/step - loss: 2.9885
Epoch 8/10
130/130 [==============================] - 63s 488ms/step - loss: 2.8581
Epoch 9/10
130/130 [==============================] - 64s 490ms/step - loss: 2.7259
Epoch 10/10
130/130 [==============================] - 64s 491ms/step - loss: 2.5906


# 문장생성 (I love~)

In [14]:
def generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you <end> '

# 회고

멋진 모델이 생성한 가사 (i love~)뒤에 오는 가사가 고작 
i love you를 만들기 위해 내가 모델을 설계했나하는 실망감이 든다.

이것은 처음부터 준비한 데이터셋들이 기존의 가사들로 이루어져서
간결한 문장을 만들어낸 것이란 예측이 든다.

이것을 해결하려면 
- 긴문장들로 이루어진 데이터셋을 준비하거나 
- 데이터 전처리 과정에서 본인은 2단어 이하의 문장들을 제거했지만 3단어 이하의 문장을 제거하는 방법도 있다.